In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tempfile import TemporaryDirectory
import os
from tqdm.notebook import tqdm
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
#Resnet implemented
vision_model = models.resnet18(weights = "IMAGENET1K_V1")
num_features = vision_model.fc.in_features
vision_model.fc = nn.Linear(num_features, 2)

vision_model = vision_model.to(device)



In [ ]:
#criterion and optimizer implemented
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vision_model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
train_dataloader =  torch.load("train_dataloader.pth")
validation_dataloader = torch.load("val_dataloader.pth")


#dataset sizes
train_loader_size = len(train_dataloader.dataset)
val_loader_size = len(validation_dataloader.dataset)

In [ ]:
print(train_loader_size)
print(val_loader_size)

In [ ]:
train_losses, val_losses = [], []
train_acc, val_acc= 0, 0

def ModelTraining(model, criterion, optimizer, num_epochs = 5):
    start = time.time()
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, "best_model_params.pt")
        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

    for epoch in range(num_epochs):
        #train mode
        model.train()
        running_loss = 0.0
        running_correct = 0

        for inputs, labels in tqdm(train_dataloader, desc="Training loop"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            #zero grad the optimizer
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            #Backpropagation
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_correct += torch.sum(preds == labels.data)

        train_loss = running_loss/len(train_dataloader.dataset)
        train_acc = running_correct.double()/train_loader_size
        #Appending loss and accuracy
        train_losses.append(train_loss)
 
        #validation phase
        model.eval()
        running_loss = 0.0
        running_correct = 0
        with torch.no_grad():
            for inputs, labels in tqdm(validation_dataloader, desc = "Validation loop"):
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                running_correct+= torch.sum(preds == labels.data)      
    
        val_loss = running_loss/len(validation_dataloader.dataset)
        val_acc = running_correct.double()/val_loader_size

        val_losses.append(val_loss)
   
        
        #print epoch stats
        print(f"Epoch {epoch + 1}/{num_epochs} - Train loss: {train_losses}, Val loss: {val_losses}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), best_model_params_path)

    time_elapsed = time.time() - start
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    print(f'Best val acc: {best_acc:.4f}')

    model.load_state_dict(torch.load(best_model_params_path))
    return model







In [ ]:
ModelTraining(vision_model, criterion=criterion, optimizer=optimizer, num_epochs = 5)